In [1]:
import requests
from bs4 import BeautifulSoup
import random
import pandas as pd

In [3]:
title = 'Dados'
location = 'Brasil'

work_model = {'1': 'Presencial', '2': 'Remoto', '3': 'Híbrido'}

In [4]:
job_ids = set()
job_work_model = {}

for n_results in range(0,21,10):
    id_WM = random.choice(list(work_model.keys()))
    list_url = f'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={title}&location=Brasil&geoId=106057199&f_WT={\
        id_WM}&position=1&start={n_results}'

    response = requests.get(list_url)

    list_data = response.text
    list_soup = BeautifulSoup(list_data, 'html.parser')
    job_cards = list_soup.find_all('div', class_='base-card')

    for card in job_cards:
        if 'data-entity-urn' in card.attrs:
            job_id = card['data-entity-urn'].split(':')[3]
            job_ids.add(job_id)
            job_work_model[job_id] = work_model[id_WM]

job_ids = list(job_ids)

In [5]:
len(job_ids)
print(job_ids)
print(len(job_ids))

['4184262596', '4191196959', '4152258042', '4179429367', '4178256022', '4184711760', '4186563017', '4192442079', '4159081209', '4146772081', '4184358084', '4185677926', '4188601092', '4189220057', '4190106005', '4182151010', '4185696461', '4075025076', '4183504583', '4189742871', '4184709447', '4142098937', '4176740814', '4170063595', '4167869097', '4181438889', '4187579023', '4188230342', '4153856219']
29


In [8]:
job_list = []

for job_id in job_ids:
    job_url = f'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}?_l=pt_BR'
    job_response = requests.get(job_url)
    # print(job_response.status_code)

    if job_response.status_code == 200:
        job_post = {}
        job_soup = BeautifulSoup(job_response.text, 'html.parser')

        job_post['job_id'] = job_id

        try:
            job_post['job_title'] = job_soup.find(
                'h2', {'class': 'top-card-layout__title'}
            ).text.strip()
        except:
            job_post['job_title'] = None

        try:
            job_post['company_name'] = job_soup.find(
                'a', {'class': 'topcard__org-name-link'}
            ).text.strip()
        except:
            job_post['company_name'] = None

        job_post['work_model'] = job_work_model.get(job_id, None)

        try:
            job_post['location'] = job_soup.find(
                'span', {'class': 'topcard__flavor topcard__flavor--bullet'}
            ).text.strip()
        except:
            job_post['location'] = None   

        try:
            job_post['time_posted'] = job_soup.find(
                'span', {'class': 'posted-time-ago__text'}
            ).text.strip()
        except:
            job_post['time_posted'] = None

        try:
            job_post['num_applicants'] = job_soup.find(
                'span', {'class': 'num-applicants__caption'}
            ).text.strip()
        except:
            job_post['num_applicants'] = None
        
        job_criteria = {}
        for item in job_soup.find_all('li', class_='description__job-criteria-item'):
            try:
                label = item.find('h3', class_='description__job-criteria-subheader').text.strip()
                value = item.find('span', class_='description__job-criteria-text').text.strip()
                job_criteria[label] = value
            except:
                continue

        job_post['xp_level'] = job_criteria.get(list(job_criteria.keys())[0], None)
        job_post['job_type'] = job_criteria.get(list(job_criteria.keys())[1], None)
        job_post['job_sectors'] = job_criteria.get(list(job_criteria.keys())[3], None)

        try:
            job_post['job_description'] = job_soup.find(
                'div', {'class': 'show-more-less-html__markup'}
            ).get_text(separator="\n", strip=True)
        except:
            job_post['job_description'] = None

        job_list.append(job_post)

In [9]:
jobs_df = pd.DataFrame(job_list)
jobs_df

,job_id,job_title,company_name,work_model,location,time_posted,num_applicants,xp_level,job_type,job_sectors,job_description
0,4184262596,Analista de Operações de Serviços Financeiros ...,Contabilizei,Remoto,Brasil,Há 1 semana,None,Júnior,Tempo integral,Atividades dos serviços de tecnologia da infor...,A sua missão:\nApoiar para a eficiência operac...
1,4191196959,Estágio em Ciência de Dados,Bemobi,Remoto,Brasil,Há 7 horas,None,Não aplicável,Estágio,Atividades dos serviços de tecnologia da infor...,Na Bemobi valorizamos o espírito inovador e co...
2,4152258042,Estágio em Análise de Dados,E,Presencial,São Paulo e Região,Há 1 mês,159 candidaturas,Estágio,Tempo integral,Serviços advocatícios,Interpretar indicadores de performance ligados...
3,4179429367,Pessoa Desenvolvedora Backend Junior,Cora,Remoto,Brasil,Há 2 semanas,None,Assistente,Tempo integral,Atividades de serviços financeiros,Sua carreira com liberdade e propósito.\nVocê ...
4,4178256022,Assistente de Contas a Receber (Foco em Cobrança),Italac,Presencial,"São Paulo, SP",Há 2 semanas,98 candidaturas,Não aplicável,Tempo integral,Fabricação de produtos alimentícios e bebidas,Realizar trabalhos referentes a baixa no siste...
5,4184711760,Banco de Talentos - Analista de Políticas de C...,Neon,Remoto,Brasil,Há 1 semana,None,Assistente,Tempo integral,Atividades de serviços financeiros,"Sobre a Neon\nQueremos ser, cada vez mais, um ..."
6,4186563017,Analista Júnior | Gestão de Carteira de Crédito,XP Inc.,Presencial,"São Paulo, SP",Há 6 dias,None,Assistente,Tempo integral,Atividades de serviços financeiros,"Sobre nós\nNossa história começou em 2001, com..."
7,4192442079,Estagiário (a) de Dados,Lojas Eskala,Presencial,"São Paulo, SP",Há 1 dia,None,Estágio,Tempo integral,Comércio varejista,"DESCRIÇÃO\nSe você é apaixonado por dados, tem..."
8,4159081209,Analista de Sucesso do Cliente - Implementação,Omie,Remoto,Brasil,Há 1 mês,None,Não aplicável,Tempo integral,Atividades dos serviços de tecnologia da infor...,A Omie tem como propósito trazer prosperidade ...
9,4146772081,Estágio em Análise de Dados | Gente e Gestão C...,Solví Soluções para a Vida,Presencial,"São Paulo, SP",Há 1 mês,None,Não aplicável,Estágio,Fabricação de máquinas e equipamentos de uso i...,Estamos em busca de uma pessoa apaixonada por ...


In [2]:
work_model_id = 'random'
n_ids = 10
WORK_MODEL = {'1': 'Presencial', '2': 'Remoto', '3': 'Híbrido'}
JOB_KEYWORD = 'Dados'

if work_model_id not in ['1', '2', '3', 'random']:
    raise ValueError("Set an appropriate value for work_model_id ('1', '2', '3', 'random')")

job_data = {}

for n_results in range(0,n_ids,10):
    current_work_model = random.choice(list(WORK_MODEL.keys())) if work_model_id == 'random' else work_model_id
    
    list_url = f'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={JOB_KEYWORD}&location=Brasil&geoId=106057199&f_WT={\
        current_work_model}&position=1&start={n_results}'

    response = requests.get(list_url)
    list_soup = BeautifulSoup(response.text, 'html.parser')
    job_cards = list_soup.find_all('div', class_='base-card')

    for card in job_cards:
        if 'data-entity-urn' in card.attrs:
            job_id = card.get('data-entity-urn', '').split(':')[-1]

            if job_id and job_id not in job_data:
                job_data[job_id] = WORK_MODEL[current_work_model]

In [7]:
job_list = []
for job_id, job_work_model in job_data.items():
    job_url = f'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}?_l=pt_BR'
    job_response = requests.get(job_url)

    if job_response.status_code == 200:
        job_post = {}
        job_soup = BeautifulSoup(job_response.text, 'html.parser')

        job_post['job_id'] = job_id

        try:
            job_post['job_title'] = job_soup.find(
                'h2', {'class': 'top-card-layout__title'}
            ).text.strip()
        except:
            job_post['job_title'] = None

        try:
            job_post['company_name'] = job_soup.find(
                'a', {'class': 'topcard__org-name-link'}
            ).text.strip()
        except:
            job_post['company_name'] = None

        job_post['work_model'] = job_work_model

        try:
            job_post['location'] = job_soup.find(
                'span', {'class': 'topcard__flavor topcard__flavor--bullet'}
            ).text.strip()
        except:
            job_post['location'] = None   

        try:
            job_post['time_posted'] = job_soup.find(
                'span', {'class': 'posted-time-ago__text'}
            ).text.strip()
        except:
            job_post['time_posted'] = None

        try:
            job_post['num_applicants'] = job_soup.find(
                'span', {'class': 'num-applicants__caption'}
            ).text.strip()
        except:
            job_post['num_applicants'] = None
        
        job_criteria = {}
        for item in job_soup.find_all('li', class_='description__job-criteria-item'):
            try:
                label = item.find('h3', class_='description__job-criteria-subheader').text.strip()
                value = item.find('span', class_='description__job-criteria-text').text.strip()
                job_criteria[label] = value
            except:
                continue

        job_post['xp_level'] = job_criteria.get(list(job_criteria.keys())[0], None)
        job_post['job_type'] = job_criteria.get(list(job_criteria.keys())[1], None)
        job_post['job_sectors'] = job_criteria.get(list(job_criteria.keys())[3], None)

        try:
            job_post['job_description'] = job_soup.find(
                'div', {'class': 'show-more-less-html__markup'}
            ).get_text(separator="\n", strip=True)
        except:
            job_post['job_description'] = None

        job_list.append(job_post)


In [3]:
df_jobs = pd.read_csv('../data/raw/jobs_data.csv', index_col=0)

In [8]:
df_jobs

,job_id,job_title,company_name,work_model,location,time_posted,num_applicants,xp_level,job_type,job_sectors,job_description
0,4150644206,Estágio Analise de Dados,EPTV,Híbrido,"Campinas, SP",Há 1 mês,NaN,Pleno-sênior,Tempo integral,Atividades de rádio e de televisão,Descrição\nEstamos em busca de um(a) estagiári...
1,4180219635,Trainee de Desenvolvimento - Matarazzo,RSM Brasil,Híbrido,"São Paulo, SP",Há 2 semanas,NaN,Não aplicável,Tempo integral,Atividades de contabilidade,Lugar de gente inovadora que faz acontecer!\nS...
2,4169347554,Analista Júnior de Produtos - Cartões,Banco Bmg,Híbrido,"São Paulo, SP",Há 6 dias,NaN,Não aplicável,Tempo integral,Bancos,"Há mais de 90 anos, o Banco Bmg vem construind..."
3,4168263642,ESTÁGIO EM BUSINESS INTELLIGENCE,Zilor,Híbrido,"São Paulo, SP",Há 4 semanas,NaN,Estágio,Estágio,Agricultura,Somos uma indústria brasileira com atuação glo...
4,4150286242,Analista Júnior de Dados - Prevenção à Fraude,Shopee,Híbrido,"São Paulo, SP",Há 2 semanas,NaN,Assistente,Tempo integral,Plataformas de comercialização na Internet e T...,"About The Team\nSobre a Shô\nAqui na Shô, acre..."
5,4075018778,Estágio Comercial REDE | Itaú Unibanco - VAGAS...,Itaú Unibanco,Híbrido,"São Paulo, SP",Há 5 dias,NaN,Assistente,Tempo integral,Bancos,Que tal construir a REDE com a gente? 🚀\nHá 26...
6,4149731885,Estágio em Sistemas da Informação,EPTV,Híbrido,"Campinas, SP",Há 1 mês,178 candidaturas,Assistente,Tempo integral,Atividades de rádio e de televisão,Descrição\nO estagiário(a) de Sistemas da Info...
7,4075025088,Faça sua Carreira de Engenharia de Processos n...,Itaú Unibanco,Híbrido,"São Paulo, SP",Há 2 semanas,NaN,Assistente,Tempo integral,Bancos,Vem construir no Itaú_\nComo maior banco priva...
8,4187888381,Estagiário de Análise de Dados,Neon Comercial Reagentes Analíticos,Híbrido,São Paulo e Região,Há 1 semana,NaN,Estágio,Estágio,Fabricação de produtos químicos,"A NEON, ciente de sua responsabilidade social ..."
9,4187544975,Analista de Dados e BI Júnior I Modelo Hibrido,Fast Shop S/A,Híbrido,"São Paulo, SP",Há 1 semana,NaN,Não aplicável,Tempo integral,Comércio varejista,Somos uma empresa brasileira de varejo reconhe...


In [7]:
df_jobs['job_description'][0]

'Descrição\nEstamos em busca de um(a) estagiário(a) para integrar nosso time de dados, focado na automatização e análise de dados, contribuindo diretamente para a tomada de decisão estratégica da empresa.\nResponsabilidades e atribuições\nAtuar na construção, governança e manutenção da arquitetura de dados do RPA.\nAutomatizar processo e otimização para reduzir esforços operacionais repetitivos.\nPromover a cultura de dados dentro da empresa e junto aos clientes, auxiliando em projetos em andamento.\nParticipar da concessão, estruturação e organização de dados.\nContribuir para projetos de Inteligência Artificial (IA).\nApoiar a construção da arquitetura de dados e o desenvolvimento de novas funcionalidades que promovem escalabilidade e eficiência.\nRequisitos e qualificações\nCursando graduação em Tecnologia, Engenharia, Estatística, Ciência de Dados ou áreas correlatas.\nNoções de SQL para manipulação e extração de dados.\nFamiliaridade com ferramentas de BI (ex.: Power BI, Metabase)